# Import Packages and Read Data

In [31]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

In [32]:
df_train = pd.read_csv('data/train_accounts.csv')
df_test = pd.read_csv('data/test_accounts.csv')
df_transactions = pd.read_csv('data/transactions.csv')

In [33]:
train_list = df_train.account.to_list()
test_list = df_test.account.to_list()
whole_list = train_list + test_list

print('Training records:', len(train_list), 'Testing records', len(test_list))
print('Total records:', len(whole_list), len(set(whole_list)))

fraud_list = df_train[df_train.flag == 1].account.to_list()
print('Fraud records:', len(fraud_list))

Training records: 25198 Testing records 6300
Total records: 31498 31498
Fraud records: 2455


In [34]:
print(df_transactions.shape)
print(df_transactions.from_account.nunique(), df_transactions.to_account.nunique())

(5826604, 6)
604847 419535


## Preprocessing

In [35]:
print(df_transactions.isnull().sum())

from_account            0
to_account              0
transaction_time_utc    0
value                   0
gas                     0
gas_price               0
dtype: int64


In [36]:
df_transactions.transaction_time_utc = pd.to_datetime(df_transactions.transaction_time_utc)
df_transactions['transaction_year'] = df_transactions.transaction_time_utc.dt.year.astype(int)
df_transactions['transaction_month_year'] = df_transactions.transaction_time_utc.dt.to_period('M')
df_transactions['transaction_date'] = df_transactions.transaction_time_utc.dt.date

In [37]:
df_transactions['gas_price'] = df_transactions['gas_price'].div(1000000000)
df_transactions['gas_fee'] = df_transactions['gas'].mul(df_transactions['gas_price'])
df_transactions['gas_fee'].describe()

count    5.826604e+06
mean     1.526253e+07
std      9.673410e+07
min      0.000000e+00
25%      4.590000e+05
50%      1.890000e+06
75%      9.000000e+06
max      6.923966e+10
Name: gas_fee, dtype: float64

In [38]:
df_transactions = df_transactions.assign(count_flag=1)
df_transactions['is_token'] = np.where(df_transactions['value']=='0', 1, 0)
df_transactions['value_digit'] = df_transactions.value.apply(lambda x: len(x))

In [39]:
df_transactions_part_1 = df_transactions.copy()

In [40]:
df_transactions.describe()

,gas,gas_price,transaction_year,gas_fee,count_flag,is_token,value_digit
count,5.826604e+06,5.826604e+06,5.826604e+06,5.826604e+06,5826604.0,5.826604e+06,5.826604e+06
mean,2.450964e+05,5.544367e+01,2.019302e+03,1.526253e+07,1.0,6.311481e-01,7.170294e+00
std,5.374690e+05,2.084319e+02,1.047990e+00,9.673410e+07,0.0,4.824937e-01,8.108089e+00
min,2.100000e+04,0.000000e+00,2.016000e+03,0.000000e+00,1.0,0.000000e+00,1.000000e+00
25%,5.000000e+04,6.000000e+00,2.019000e+03,4.590000e+05,1.0,0.000000e+00,1.000000e+00
50%,9.000000e+04,2.000000e+01,2.020000e+03,1.890000e+06,1.0,1.000000e+00,1.000000e+00
75%,2.500000e+05,6.000000e+01,2.020000e+03,9.000000e+06,1.0,1.000000e+00,1.700000e+01
max,1.202223e+07,1.713970e+05,2.021000e+03,6.923966e+10,1.0,1.000000e+00,2.300000e+01


# Dataset Construction

## Feature Engineering

1. from account features
2. to account features

- maximum, mean, std transaction number within the same date / month / year
- total number of transaction
- year count
- from/to account identified as fraud count
- total number of token/non-token transaction
- max, min, mean, std of gas price / gas / gas_fee
- max, min, mean, std of transaction value_digit

In [41]:
df_transactions.columns

Index(['from_account', 'to_account', 'transaction_time_utc', 'value', 'gas',
       'gas_price', 'transaction_year', 'transaction_month_year',
       'transaction_date', 'gas_fee', 'count_flag', 'is_token', 'value_digit'],
      dtype='object')

In [42]:
df_all = pd.DataFrame(data={'account': whole_list})
df_all = df_all.assign(src_dst_account_fraud_or_not=0)

usable_var = ['transaction_date', 'transaction_year', 'transaction_month_year',
              'gas', 'gas_price', 'gas_fee', 'is_token', 'value_digit', 'count_flag']

for var in ['src', 'dst']:
    if var == 'src':
        account_feature = 'from_account'
        counter_feature = 'to_account'
    else:
        account_feature = 'to_account'
        counter_feature = 'from_account'

    # maximum, mean, std transaction number within the same date / month / year
    for criteria in ['transaction_date', 'transaction_month_year', 'transaction_year']:
        df_temp = df_transactions.groupby([account_feature, criteria]).count_flag.count()
        df_temp_max = df_temp.reindex().groupby([account_feature]).max()
        df_temp_max = df_temp_max.rename(f'{var}_max_records_by_{criteria}').reset_index()
        df_temp_mean = df_temp.reindex().groupby([account_feature]).mean()
        df_temp_mean = df_temp_mean.rename(f'{var}_mean_records_by_{criteria}').reset_index()
        df_temp_std = df_temp.reindex().groupby([account_feature]).std(ddof=0)
        df_temp_std = df_temp_std.rename(f'{var}_std_records_by_{criteria}').reset_index()

        df_all = df_all.merge(df_temp_max, how='left', left_on='account',
                              right_on=account_feature).drop(columns=account_feature)
        df_all = df_all.merge(df_temp_mean, how='left', left_on='account',
                              right_on=account_feature).drop(columns=account_feature)
        df_all = df_all.merge(df_temp_std, how='left', left_on='account',
                              right_on=account_feature).drop(columns=account_feature)
    df_all = df_all.fillna(0)

    # total number of transaction
    df_temp_tot = df_transactions.groupby([account_feature]).count_flag.count()
    df_temp_tot = df_temp_tot.rename(f'{var}_total_transactions').reset_index()
    df_all = df_all.merge(df_temp_tot, how='left', left_on='account',
                          right_on=account_feature).drop(columns=account_feature)
    df_all = df_all.fillna(0)

    # year count
    df_temp_year = df_transactions.groupby([account_feature]).transaction_year.nunique()
    df_temp_year = df_temp_year.rename(f'{var}_year_count').reset_index()
    df_all = df_all.merge(df_temp_year, how='left', left_on='account',
                          right_on=account_feature).drop(columns=account_feature)
    df_all = df_all.fillna(0)

    # from/to account identified as fraud count: 0 is unknown / not fraud
    df_temp_fraud = df_transactions.merge(df_train, how='left', left_on=counter_feature,
                                          right_on='account').drop(columns='account')
    df_temp_fraud = df_temp_fraud.groupby([account_feature]).flag.max()
    df_temp_fraud = df_temp_fraud.rename(f'flag_{var}').reset_index().fillna(0)
    df_all = df_all.merge(df_temp_fraud, how='left', left_on='account',
                          right_on=account_feature).drop(columns=account_feature)
    df_all['src_dst_account_fraud_or_not'] = np.where(df_all[f'flag_{var}']==1,
                                                      1, df_all['src_dst_account_fraud_or_not'])

    # total number of token/non-token transaction
    df_temp_token = df_transactions.groupby([account_feature]).is_token.sum()
    df_temp_token = df_temp_token.rename(f'{var}_token_record_count').reset_index()
    df_all = df_all.merge(df_temp_token, how='left', left_on='account',
                          right_on=account_feature).drop(columns=account_feature)
    df_all = df_all.fillna(0)

    # max, min, mean, std of gas price / gas / gas_fee
    for feature in ['gas', 'gas_price', 'gas_fee']:
        df_temp_max = df_transactions.groupby([account_feature])[feature].max()
        df_temp_max = df_temp_max.rename(f'{var}_max_{feature}').reset_index()
        df_temp_min = df_transactions.groupby([account_feature])[feature].min()
        df_temp_min = df_temp_min.rename(f'{var}_min_{feature}').reset_index()
        df_temp_mean = df_transactions.groupby([account_feature])[feature].mean()
        df_temp_mean = df_temp_mean.rename(f'{var}_mean_{feature}').reset_index()
        df_temp_std = df_transactions.groupby([account_feature])[feature].std(ddof=0)
        df_temp_std = df_temp_std.rename(f'{var}_std_{feature}').reset_index()

        df_all = df_all.merge(df_temp_max, how='left', left_on='account',
                              right_on=account_feature).drop(columns=account_feature)
        df_all = df_all.merge(df_temp_min, how='left', left_on='account',
                              right_on=account_feature).drop(columns=account_feature)
        df_all = df_all.merge(df_temp_mean, how='left', left_on='account',
                              right_on=account_feature).drop(columns=account_feature)
        df_all = df_all.merge(df_temp_std, how='left', left_on='account',
                              right_on=account_feature).drop(columns=account_feature)
    df_all = df_all.fillna(0)

    # max, min, mean, std of transaction value_digit
    for feature in ['value_digit']:
        df_temp_max = df_transactions.groupby([account_feature])[feature].max()
        df_temp_max = df_temp_max.rename(f'{var}_max_{feature}').reset_index()
        df_temp_min = df_transactions.groupby([account_feature])[feature].min()
        df_temp_min = df_temp_min.rename(f'{var}_min_{feature}').reset_index()
        df_temp_mean = df_transactions.groupby([account_feature])[feature].mean()
        df_temp_mean = df_temp_mean.rename(f'{var}_mean_{feature}').reset_index()
        df_temp_std = df_transactions.groupby([account_feature])[feature].std(ddof=0)
        df_temp_std = df_temp_std.rename(f'{var}_std_{feature}').reset_index()

        df_all = df_all.merge(df_temp_max, how='left', left_on='account',
                              right_on=account_feature).drop(columns=account_feature)
        df_all = df_all.merge(df_temp_min, how='left', left_on='account',
                              right_on=account_feature).drop(columns=account_feature)
        df_all = df_all.merge(df_temp_mean, how='left', left_on='account',
                              right_on=account_feature).drop(columns=account_feature)
        df_all = df_all.merge(df_temp_std, how='left', left_on='account',
                              right_on=account_feature).drop(columns=account_feature)
    df_all = df_all.fillna(0)

In [43]:
df_all_part2 = df_all.copy()
df_all_part2.to_pickle('data/df_all_part2.pkl')

In [44]:
display(df_all.isnull().mean().to_frame(name='null_pct').sort_values('null_pct', ascending=False).T)
display(df_all.describe())

,account,src_dst_account_fraud_or_not,dst_mean_records_by_transaction_date,dst_std_records_by_transaction_date,dst_max_records_by_transaction_month_year,dst_mean_records_by_transaction_month_year,dst_std_records_by_transaction_month_year,dst_max_records_by_transaction_year,dst_mean_records_by_transaction_year,dst_std_records_by_transaction_year,...,src_max_gas_price,src_min_gas_price,src_mean_gas_price,src_std_gas_price,src_max_gas_fee,src_min_gas_fee,src_mean_gas_fee,src_std_gas_fee,src_max_value_digit,dst_std_value_digit
null_pct,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,src_dst_account_fraud_or_not,src_max_records_by_transaction_date,src_mean_records_by_transaction_date,src_std_records_by_transaction_date,src_max_records_by_transaction_month_year,src_mean_records_by_transaction_month_year,src_std_records_by_transaction_month_year,src_max_records_by_transaction_year,src_mean_records_by_transaction_year,src_std_records_by_transaction_year,...,dst_mean_gas_price,dst_std_gas_price,dst_max_gas_fee,dst_min_gas_fee,dst_mean_gas_fee,dst_std_gas_fee,dst_max_value_digit,dst_min_value_digit,dst_mean_value_digit,dst_std_value_digit
count,31498.000000,31498.000000,31498.000000,31498.000000,31498.000000,31498.000000,31498.000000,31498.000000,31498.000000,31498.000000,...,31498.000000,31498.000000,3.149800e+04,3.149800e+04,3.149800e+04,3.149800e+04,31498.000000,31498.000000,31498.000000,31498.000000
mean,0.022954,7.625056,2.214283,1.375487,28.888564,11.281139,8.700305,66.339450,43.096857,19.782068,...,44.848855,21.770248,1.103431e+07,1.051480e+06,2.612086e+06,1.972031e+06,17.606515,15.090799,16.636491,0.797434
std,0.149759,77.702994,9.856640,11.712608,351.726158,115.172637,114.144330,708.100917,528.918592,233.016010,...,56.288204,43.749636,8.235071e+07,4.148342e+06,7.441957e+06,7.895046e+06,3.651680,5.356001,3.814936,1.634922
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,2.000000,2.000000,0.000000,...,10.500000,0.000000,5.670000e+05,6.300000e+04,3.832634e+05,0.000000e+00,17.000000,16.000000,16.500000,0.000000
50%,0.000000,2.000000,1.166667,0.205381,4.000000,2.500000,0.707107,6.000000,5.000000,0.000000,...,26.439062,7.850542,2.205259e+06,2.625000e+05,1.092000e+06,3.551181e+05,18.000000,17.000000,17.500000,0.427790
75%,0.000000,4.000000,2.000000,0.829156,10.000000,5.000000,2.943920,22.000000,15.666667,3.741657,...,55.434643,25.981440,7.500000e+06,8.781150e+05,2.566087e+06,1.661411e+06,19.000000,18.000000,18.250000,0.742307
max,1.000000,6308.000000,854.777778,1062.457950,40397.000000,8366.142857,13772.158389,58563.000000,58563.000000,12085.018628,...,1210.420130,2144.796031,6.492470e+09,2.000000e+08,3.107304e+08,5.918665e+08,23.000000,23.000000,23.000000,10.000000


## EDA

- explore distribution: implement log transformation
- profiling, correlation plot

In [45]:
print(df_all.columns)

Index(['account', 'src_dst_account_fraud_or_not',
       'src_max_records_by_transaction_date',
       'src_mean_records_by_transaction_date',
       'src_std_records_by_transaction_date',
       'src_max_records_by_transaction_month_year',
       'src_mean_records_by_transaction_month_year',
       'src_std_records_by_transaction_month_year',
       'src_max_records_by_transaction_year',
       'src_mean_records_by_transaction_year',
       'src_std_records_by_transaction_year', 'src_total_transactions',
       'src_year_count', 'flag_src', 'src_token_record_count', 'src_max_gas',
       'src_min_gas', 'src_mean_gas', 'src_std_gas', 'src_max_gas_price',
       'src_min_gas_price', 'src_mean_gas_price', 'src_std_gas_price',
       'src_max_gas_fee', 'src_min_gas_fee', 'src_mean_gas_fee',
       'src_std_gas_fee', 'src_max_value_digit', 'src_min_value_digit',
       'src_mean_value_digit', 'src_std_value_digit',
       'dst_max_records_by_transaction_date',
       'dst_mean_records_by_t

In [46]:
df_all = pd.read_pickle('data/df_all_part2.pkl')

usable_var = ['account', 'src_dst_account_fraud_or_not',
              'src_max_records_by_transaction_date',
              'src_mean_records_by_transaction_date',
              'src_std_records_by_transaction_date',
              'src_max_records_by_transaction_month_year',
              'src_mean_records_by_transaction_month_year',
              'src_std_records_by_transaction_month_year',
              'src_max_records_by_transaction_year',
              'src_mean_records_by_transaction_year',
              'src_std_records_by_transaction_year',
              'src_total_transactions', 'src_year_count',
              'src_token_record_count',
              'src_max_gas', 'src_min_gas', 'src_mean_gas', 'src_std_gas',
              'src_max_gas_price', 'src_min_gas_price',
              'src_mean_gas_price', 'src_std_gas_price',
              'src_max_gas_fee', 'src_min_gas_fee',
              'src_mean_gas_fee', 'src_std_gas_fee',
              'src_max_value_digit', 'src_min_value_digit',
              'src_mean_value_digit', 'src_std_value_digit',
              'dst_max_records_by_transaction_date',
              'dst_mean_records_by_transaction_date',
              'dst_std_records_by_transaction_date',
              'dst_max_records_by_transaction_month_year',
              'dst_mean_records_by_transaction_month_year',
              'dst_std_records_by_transaction_month_year',
              'dst_max_records_by_transaction_year',
              'dst_mean_records_by_transaction_year',
              'dst_std_records_by_transaction_year',
              'dst_total_transactions', 'dst_year_count',
              'dst_token_record_count',
              'dst_max_gas', 'dst_min_gas', 'dst_mean_gas', 'dst_std_gas',
              'dst_max_gas_price', 'dst_min_gas_price',
              'dst_mean_gas_price', 'dst_std_gas_price',
              'dst_max_gas_fee', 'dst_min_gas_fee',
              'dst_mean_gas_fee', 'dst_std_gas_fee',
              'dst_max_value_digit', 'dst_min_value_digit',
              'dst_mean_value_digit', 'dst_std_value_digit']

log_var = ['src_max_gas', 'src_min_gas',
           'src_mean_gas', 'src_std_gas',
           'src_max_gas_price', 'src_min_gas_price',
           'src_mean_gas_price', 'src_std_gas_price',
           'src_max_gas_fee', 'src_min_gas_fee',
           'src_mean_gas_fee', 'src_std_gas_fee',
           'dst_max_gas', 'dst_min_gas',
           'dst_mean_gas', 'dst_std_gas',
           'dst_max_gas_price', 'dst_min_gas_price',
           'dst_mean_gas_price', 'dst_std_gas_price',
           'dst_max_gas_fee', 'dst_min_gas_fee',
           'dst_mean_gas_fee', 'dst_std_gas_fee']

df_all = df_all[usable_var]

In [47]:
df_all[log_var].isnull().mean()

src_max_gas           0.0
src_min_gas           0.0
src_mean_gas          0.0
src_std_gas           0.0
src_max_gas_price     0.0
src_min_gas_price     0.0
src_mean_gas_price    0.0
src_std_gas_price     0.0
src_max_gas_fee       0.0
src_min_gas_fee       0.0
src_mean_gas_fee      0.0
src_std_gas_fee       0.0
dst_max_gas           0.0
dst_min_gas           0.0
dst_mean_gas          0.0
dst_std_gas           0.0
dst_max_gas_price     0.0
dst_min_gas_price     0.0
dst_mean_gas_price    0.0
dst_std_gas_price     0.0
dst_max_gas_fee       0.0
dst_min_gas_fee       0.0
dst_mean_gas_fee      0.0
dst_std_gas_fee       0.0
dtype: float64

In [48]:
# profile = ProfileReport(df_all, title="Feature Report")
# profile.to_file("data/feature_profile.html")

In [50]:
for feature in log_var:
    df_all[feature] = np.log1p(df_all[feature])

df_all_part3 = df_all.copy()
df_all_part3.to_pickle('data/df_all_part2.pkl')

# Modelling

- implement models: regression, autoencoder, boosting, RNN

In [ ]:
X_train = df_all[df_all.account.isin(train_list)].sort_values(by='account').reset_index(drop=True)
X_test = df_all[df_all.account.isin(test_list)].sort_values(by='account').reset_index(drop=True)